In [1]:
import pandas as pd

In [2]:
data_dir = "data/a/combined_data_raw.csv"

In [22]:
df = pd.read_csv(data_dir, index_col=0)

In [25]:
# Check for missing values
print(f"Number of rows with missing values: {df.isnull().sum()}")

# Check for duplicates
print(f"Number of duplicate rows: {df.duplicated(subset=['statement']).sum()}")

# Check the first few rows for structure
print(df.head())

Number of rows with missing values: statement    362
status         0
dtype: int64
Number of duplicate rows: 1969
                                           statement   status
0                                         oh my gosh  Anxiety
1  trouble sleeping, confused mind, restless hear...  Anxiety
2  All wrong, back off dear, forward doubt. Stay ...  Anxiety
3  I've shifted my focus to something else but I'...  Anxiety
4  I'm restless and restless, it's been a month n...  Anxiety


In [26]:
df = df.dropna()
df = df.drop_duplicates(subset=['statement'], keep='first')
df.reset_index(drop = True, inplace = True)

In [27]:
df.status.value_counts()

status
Normal                  16039
Depression              15087
Suicidal                10641
Anxiety                  3617
Bipolar                  2501
Stress                   2293
Personality disorder      895
Name: count, dtype: int64

In [28]:
df.to_csv('data/a/cleaned_data.csv', index=False)